In [1]:
import pandas as pd
import base64
import re

def decode_base64(word):
    try:
        return base64.b64decode(word).decode()
    except:
        pass

with open("commands.txt", "r") as f:
    vocabulary = f.read().splitlines()


def split_by_vocabulary(string):
    pattern = '|'.join(map(re.escape, vocabulary))
    matches = re.findall(pattern, string)
    return matches


def decode_session(full_session):
    new_full_session = []
    
    for session_chunk in full_session.split(";"):
        if "base64 --decode" in session_chunk and "echo" in session_chunk:
            parts = session_chunk.split()
            base64_encoded = None
        
            for i in range(len(parts)):
                if parts[i] == "echo":
                    base64_encoded = parts[i + 1].strip("\"")
                    break
            
            if base64_encoded:
                decoded = decode_base64(base64_encoded)
                if decoded:
                    words_decoded = decoded.split("\n")
                    # Remove shebang if present
                    if words_decoded[0].strip() == "#!/bin/bash":
                        words_decoded[0] = ""
                    new_full_session.append("; ".join(list(filter(None, words_decoded))).strip())
                    global base64_decoded_counter
                    base64_decoded_counter += 1
                else:
                    new_full_session.append(session_chunk.strip())
            else:
                new_full_session.append(session_chunk.strip())
        else:
            new_full_session.append(session_chunk.strip())
    
    return "; ".join(new_full_session)

In [2]:
df_original = pd.read_parquet('ssh_attacks.parquet')
df_decoded=df_original.copy()
df_decoded['first_timestamp'] = pd.to_datetime(df_decoded['first_timestamp'])
global base64_decoded_counter
base64_decoded_counter = 0
df_decoded["full_session"] = df_decoded["full_session"].apply(lambda session: decode_session(session))
df_decoded.to_parquet("ssh_attacks_decoded.parquet")



In [3]:
df_decoded_splitted = df_decoded.copy()
df_decoded_splitted["full_session"] = df_decoded_splitted["full_session"].apply(lambda session: split_by_vocabulary(session))
df_decoded_splitted.to_parquet("ssh_attacks_decoded_splitted.parquet")